<a href="https://colab.research.google.com/github/nedelculiviu95/cnn-detecting-lung-opacities/blob/master/prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install pydicom
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import numpy as np
import tensorflow as tf
import pandas as pd
import cv2
import glob
import pylab
import matplotlib.pyplot as plt
import random
from random import randint
from pathlib import Path
import os
import pydicom
from skimage import measure
from skimage.transform import resize
from tensorflow import keras
import keras
import csv
from keras.callbacks import *
from os.path import isfile, join
from PIL import Image, ImageFile
import matplotlib.image as mpimg

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Using TensorFlow backend.


In [0]:
LR = 0.001
EPOCHS = 20
BATCHSIZE = 16
CHANNELS = 64
IMAGE_SIZE = 224
NBLOCK = 6 
DEPTH = 6
MOMENTUM = 0.9
DIMS = (IMAGE_SIZE,IMAGE_SIZE,3)

In [0]:
images1_folder = "/content/gdrive/My Drive/images1/"
images3_folder = "/content/gdrive/My Drive/images/"
filenames_3 = [f for f in os.listdir(images3_folder) if isfile(join(images3_folder, f))]
filenames_1 = [f for f in os.listdir(images1_folder) if isfile(join(images1_folder, f))]

train_filenames = []
folder = "/content/gdrive/My Drive/data_filenames/train.txt"
file1 = open(folder, "r") 
for line in file1: 
  l = line[0:16]
  train_filenames.append(l)
file1.close()

print('n train samples', len(train_filenames))

test_filenames = []
folder = "/content/gdrive/My Drive/data_filenames/test.txt"
file1 = open(folder, "r") 
for line in file1: 
  l = line[0:16]
  test_filenames.append(l)
file1.close()

print('n test samples', len(test_filenames))

valid_filenames = []
folder = "/content/gdrive/My Drive/data_filenames/validation.txt"
file1 = open(folder, "r") 
for line in file1: 
  l = line[0:16]
  valid_filenames.append(l)
file1.close()

print('n validation samples', len(valid_filenames))


n train samples 11665
n test samples 2729
n validation samples 1500


In [0]:
def parse_data(df):
    parsed = {}
    for n, row in df.iterrows():
        # --- Initialize patient entry into parsed 
        pid = row['Image Index']
        if pid not in parsed:
            labels = row['Finding Labels']
            parsed[pid] = {
                'png': '%s' % pid,
                'label': labels.split("|"),
                'patient_gender': row['Patient Gender'],
                'pattient_age': row['Patient Age']}
        
        #labels = row['Finding Labels']
        #parsed[pid]['label'].append(labels.split("|"))
        
    return parsed
  
  
df = pd.read_csv('/content/gdrive/My Drive/ChestXRay_csv/Data_Entry_2017.csv')
parsed = parse_data(df)


In [0]:
def concrete_labels(parsed, filenames):
  n_labels = 14
  labels_array = np.zeros(len(filenames), dtype=int) #4999 x 14
  dictionary = {'Atelectasis': 0, 'Cardiomegaly': 1, 'Effusion': 2, 'Infiltration':3, 
                'Mass': 4, 'Nodule': 5, 'Pneumonia': 6, 'Pneumothorax': 7, 'Consolidation': 8,
               'Edema':9, 'Emphysema':10, 'Fibrosis': 11, 'Pleural_Thickening': 12, 'Hernia': 13}
  i = 0
  target = 1
  for item in filenames:
    diseases = parsed[item]['label']
    for d in diseases:
      if d == 'No Finding':
        target = 0
        break
      else:
        target = 1
        break
    labels_array.append(target)
   
  return labels_array

def get_label(filename, parsed):
  n_labels = 14
  label_array = np.zeros((1, n_labels), dtype=int) #1 x 14
  diseases = parsed[filename]['label']
  target = 1
  for d in diseases:
      if d == 'No Finding':
        target = 0
        break
      else:
        target = 1
        break
  return target

yy = get_label(train_filenames[2], parsed)
print("filename: ", train_filenames[2] )
print("label: ", yy )

filename:  00003923_016.png
label:  0


In [0]:
##data generator
labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 
                'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation',
               'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']
class data_generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, parsed, batch_size=BATCHSIZE, image_size=IMAGE_SIZE, shuffle=True, augment=False, predict=False):
        self.folder = folder
        self.filenames = filenames
        self.parsed = parsed
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.augment = augment
        self.predict = predict
        self.on_epoch_end()
    
    def __load__(self, filename):
        img_path = os.path.join(self.folder, filename)
        img = Image.open(img_path)
        img_arr = np.asarray(img)
        label = 1
        diseases = parsed[filename]['label']
        if diseases[0] == 'No Finding':
            label = 0
        img_arr = resize(img_arr, (self.image_size, self.image_size), mode='reflect')
        if self.augment and random.random() > 0.5:
            img = np.fliplr(img)=
        img = np.expand_dims(img_arr, -1)
        return img, label
      
    def __loadpredict__(self, filename):
        img_path = os.path.join(self.folder, filename)
        img = Image.open(img_path)
        img_arr = np.asarray(img)
        img = resize(img_arr, (self.image_size, self.image_size), mode='reflect')
        img = np.expand_dims(img, -1)
        return img, label
      
    def __getitem__(self, index):
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        if self.predict:
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            imgs = np.array(imgs)
            return imgs, filenames
        else:=
            items = [self.__load__(filename) for filename in filenames]
            imgs, labels = zip(*items)
            imgs = np.array(imgs)
            labels = np.array(labels)
            return imgs, labels
    def on_epoch_end(self,):
        if self.shuffle:
            random.shuffle(self.filenames)
    def __len__(self):
        if self.predict:
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            return int(len(self.filenames) / self.batch_size)

In [0]:
## create model
from keras.layers import BatchNormalization, ReLU, Conv2D, AveragePooling2D, MaxPool2D, GlobalAveragePooling2D, Dense

def pooling_block(inputs, pool=2):
    x = BatchNormalization(momentum=MOMENTUM)(inputs)
    x = ReLU(0)(x)
    x = MaxPool2D(pool)(x)
    return x
  
def composite_function(channels, inputs, size=3, padding='same'):
    x = BatchNormalization(momentum=MOMENTUM)(inputs)
    x = ReLU(0)(x)
    x = Conv2D(channels, size, padding=padding, use_bias=False)(x)
    return x

def conv_block(inputs, channels1, channels2):
    x = composite_function(channels1, inputs)
    x = composite_function(channels2, x)
    x = keras.layers.Concatenate()([inputs, x])
    return x
  
def dense_block(inputs, nblocks=6, channels1=128, channels2=32):
    x = inputs
    for i in range(nblocks):
        x = conv_block(x, channels1, channels2)
    return x

def transition_block(inputs, channels, pool=2):
    x = composite_function(channels, inputs)
    x = AveragePooling2D(pool)(x)
    return x

def build_model(input_size, channels=64, channels2=32, n_blocks=NBLOCK, depth=DEPTH):
    input1 = keras.Input(shape=(input_size, input_size, 1))
    x = Conv2D(channels, 3, padding='same', strides=2, use_bias=False)(input1)
    x = pooling_block(x)
    nchan = channels
    for d in range(depth-3):
        x = dense_block(x)
        nchan = ( nchan + n_blocks*channels2 ) // 2
        x = transition_block(x, nchan)
    x = dense_block(x)
    x = GlobalAveragePooling2D()(x)
    x = (0)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs=input1, outputs=output)
    return model

In [0]:
# create network and compiler
model = build_model(input_size=IMAGE_SIZE, channels=CHANNELS, n_blocks=NBLOCK, depth=DEPTH)
#103 layers
model.summary()

model.compile(optimizer=keras.optimizers.Adam(lr=LR),
              loss=keras.losses.binary_crossentropy, metrics=['accuracy'])
train_gen = generator(images3_folder, train_filenames, parsed, batch_size=BATCHSIZE, 
                      image_size=IMAGE_SIZE, shuffle=True, augment=True, predict=False)
valid_gen = generator(images1_folder, valid_filenames, parsed, batch_size=BATCHSIZE,
                      image_size=IMAGE_SIZE, shuffle=False, predict=False)

history = model.fit_generator(train_gen, validation_data=valid_gen, 
                              epochs=EPOCHS, shuffle=True, verbose=2)

# Testing 

##1)Testing one image 

In [0]:
#testing if img has pneumonia or not 
feature = model.predict(img)
print(feature)
if feature[0][0] > 0.5:
  print("Prediction: patient",filename, "has pneumonia")
else:
  print("Prediction: patient",filename, "doesn't have pneumonia")

[[0.21551116]]
Prediction: patient 7dfe6d14-a31c-483a-be83-7a705e45719f doesn't have pneumonia


#Plotting training and validation matrics

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()